In [249]:
import urllib.request
from bs4 import BeautifulSoup
import os
from joblib import Parallel, delayed
import multiprocessing
from time import time
import requests 
import urllib.request
import json
import csv  

In [2]:
def get_text_html(link):
    return BeautifulSoup(requests.get(url=link).text, 'html.parser')

In [3]:
url = "https://www.gutenberg.org/browse/authors/a"

In [4]:
page = get_text_html(url)

In [5]:
def get_pages_by_author(data):
    pages = {}
    name_author = ''
    for item in data:
        tag_name = item.name
        if tag_name == 'h2':
            name_author = item.text.replace(' ¶','')
#             print(name_author)
        elif tag_name == 'ul':
            name_book = item.text
            links_book = []
            for i in item.findChildren('li', recurdive=False):
                if 'English' in i.text:
                    links_on_book = i.find_all('a')
                    for link in links_on_book:
                        if 'ebooks' in link['href']:
                            links_book.append(f"https://www.gutenberg.org{link['href']}")
            if len(links_book) > 0:
                pages[name_author] = links_book
    return pages

In [6]:
def get_direct_reference(link):
    result = 'empty'
    page = get_text_html(link)
    data = page.find("table", {"class": "files"}).findChildren('tr')
    for i in data:
        attributes = i.attrs
        if 'about' in attributes:
            if 'txt' in i['about']:
                result = i['about']
    return result

In [7]:
def download_by_link(link):
    urllib.request.urlretrieve(keu_for_download, dir_name + '/' + link[key])

In [8]:
pages = get_pages_by_author(page.find("div", {"class": "pgdbbyauthor"}).find('div').children)

In [19]:
def get_linc_from_dict(pages):
    count = 1
    for key in pages.keys():
        a_pool = multiprocessing.Pool()
        res = a_pool.map(get_direct_reference, pages[key])
        print(res)
        if count <= 10:
            count += 1
        else:
            break
#         for page_book in pages[key]:
#             get_direct_reference(page_book)

In [20]:
get_linc_from_dict(pages)

['https://www.gutenberg.org/ebooks/29666.txt.utf-8']
['https://www.gutenberg.org/ebooks/61217.txt.utf-8']
['https://www.gutenberg.org/ebooks/26348.txt.utf-8']
['https://www.gutenberg.org/ebooks/11861.txt.utf-8', 'https://www.gutenberg.org/ebooks/25753.txt.utf-8']
['https://www.gutenberg.org/ebooks/10338.txt.utf-8']
['https://www.gutenberg.org/ebooks/16791.txt.utf-8']
['https://www.gutenberg.org/ebooks/67167.txt.utf-8', 'https://www.gutenberg.org/files/62187/62187-0.txt']
['https://www.gutenberg.org/ebooks/23037.txt.utf-8']
['https://www.gutenberg.org/ebooks/17189.txt.utf-8']
['https://www.gutenberg.org/ebooks/19768.txt.utf-8']
['https://www.gutenberg.org/files/58663/58663-0.txt']


In [152]:
def get_dictionary(tags):
    name = ''
    result = {}
    for tag in tags:
        if tag.name == 'th':
            name = tag.text
        elif tag.name == 'td' and name != '':
            result = {name: tag.text.rstrip().lstrip()}
    return result

In [ ]:
isAuthor = 'Author' in dictionary
            isTitle = 'Title' in dictionary

In [220]:
def get_list_dictionary(link):
    list_info = []
    page = get_text_html(link)
    data = page.find("table", {"class": "bibrec"}).find_all('tr')
    for tr in data:
        dictionary = get_dictionary(tr)
#         print(dictionary)
        if len(dictionary) > 0:
            res = 'Downloads' in dictionary
            if res == False:
                list_info.append(dictionary)
    return list_info

In [221]:
def delete_duplicate(list_dict):
    keys = []
    for item in list_dict:
        keys.append(list(item.keys())[0])
    unique_keys = set(keys)
    new_dict = {}
    for key in unique_keys:
        new_list = []
        for item in list_dict:
            #print(key, item, key in item)
            if key in item:
                new_list.append(item[key])
        new_dict[key] = new_list
    return new_dict

In [238]:
def get_raw_data(pages):
    print(len(pages))
    list_dictionaty_without_duplicate = []
    count = 0
    for key in pages.keys():
        print(count, key)
        for item in pages[key]:
            list_dictionary = get_list_dictionary(item)
            list_dictionaty_without_duplicate.append(delete_duplicate(list_dictionary))
        count += 1
        if count > 50:
            break
    return list_dictionaty_without_duplicate

In [239]:
raw_data = get_raw_data(pages)

924
0 Aaberg, J. C. (Jens Christian), 1877-1970
1 Aandahl, Vance, 1942-
2 Aanrud, Hans, 1863-1953
3 Aaron, S. F. (Samuel Francis), 1862-
4 Aaronsohn, Alexander, 1888-1948
5 Abbey, Charles J. (Charles John), 1833-1919
6 Abbey, Edwin Austin, 1852-1911
7 Abbey, Henry, 1842-1911
8 Abbot, Anne W. (Anne Wales), 1808-1908
9 Abbot, Francis Ellingwood, 1836-1903
10 Abbot, Robert, cook
11 Abbott, Angus Evan, 1862-1923
12 Abbott, Belle K. (Belle Kendrick), 1842-1893
13 Abbott, Charles C. (Charles Conrad), 1843-1919
14 Abbott, David P. (David Phelps), 1863-1934
15 Abbott, Edwin Abbott, 1838-1926
16 Abbott, Eleanor Hallowell, 1872-1958
17 Abbott, Elenore Plaisted, 1876-1935
18 Abbott, Ernest Hamlin, 1870-1931
19 Abbott, Evelyn, 1843-1901
20 Abbott, Frank Frost, 1860-1924
21 Abbott, Franklin Pierce
22 Abbott, G. F. (George Frederick), 1874-1947
23 Abbott, Henry, 1850-1943
24 Abbott, Jacob, 1803-1879
25 Abbott, Jane, 1881-1968
26 Abbott, John S. C. (John Stevens Cabot), 1805-1877
27 Abbott, Keene, 18

In [244]:
def get_json(dictionary):
    return json.dumps(dictionary, indent = 0)

In [247]:
# for item in raw_data:
#     print(get_json(item))
#     break

In [273]:
def create_csv(name, data):
    header = ['Author', 'Title']
    with open(name, 'w', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for i in data:
            if 'Author' in i:
                for author in i['Author']:
                    for title in i['Title']:
                        writer.writerow([author, title])
            if 'Contributor' in i:
                for contributor in i['Contributor']:
                    for title in i['Title']:
                        writer.writerow([contributor, title])

create_csv('parsed_data.csv', raw_data)

In [228]:
def make_data_for_DB(pages, ):
    raw_data = 
    unique_keys = []
    
    for i in list_dictionaty_without_duplicate:
        for j in i.keys():
            print(j, i[j])
            res = j in unique_keys
            if res == False: 
                unique_keys.append(j)
        print()
    unique_keys.sort()
    for i in unique_keys:
        print(i)
make_data_for_DB(pages)

{'Aaberg, J. C. (Jens Christian), 1877-1970': ['https://www.gutenberg.org/ebooks/29666'], 'Aandahl, Vance, 1942-': ['https://www.gutenberg.org/ebooks/61217'], 'Aanrud, Hans, 1863-1953': ['https://www.gutenberg.org/ebooks/26348'], 'Aaron, S. F. (Samuel Francis), 1862-': ['https://www.gutenberg.org/ebooks/11861', 'https://www.gutenberg.org/ebooks/25753'], 'Aaronsohn, Alexander, 1888-1948': ['https://www.gutenberg.org/ebooks/10338'], 'Abbey, Charles J. (Charles John), 1833-1919': ['https://www.gutenberg.org/ebooks/16791'], 'Abbey, Edwin Austin, 1852-1911': ['https://www.gutenberg.org/ebooks/67167', 'https://www.gutenberg.org/ebooks/62187'], 'Abbey, Henry, 1842-1911': ['https://www.gutenberg.org/ebooks/23037'], 'Abbot, Anne W. (Anne Wales), 1808-1908': ['https://www.gutenberg.org/ebooks/17189'], 'Abbot, Francis Ellingwood, 1836-1903': ['https://www.gutenberg.org/ebooks/19768'], 'Abbot, Robert, cook': ['https://www.gutenberg.org/ebooks/58663'], 'Abbott, Angus Evan, 1862-1923': ['https://www

Author ['Aaberg, J. C. (Jens Christian), 1877-1970']
EBook-No. ['29666']
Release Date ['Aug 11, 2009']
Title ['Hymns and Hymnwriters of Denmark']
LoC Class ['BV: Philosophy, Psychology, Religion: Christianity: Practical theology, Worship']
Copyright Status ['Public domain in the USA.']
Language ['English']
Subject ['Hymns, Danish -- History and criticism', 'Hymns, Danish -- Translations into English']
Category ['Text']
LoC No. ['45006950']

Author ['Aandahl, Vance, 1942-']
EBook-No. ['61217']
Release Date ['Jan 22, 2020']
Title ['1,492,633 Marlon Brandos']
LoC Class ['PS: Language and Literatures: American and Canadian literature']
Copyright Status ['Public domain in the USA.']
Language ['English']
Subject ['Science fiction', 'Short stories', 'Riots -- Fiction', 'Men -- Fiction']
Category ['Text']

Uniform Title ['Sidsel Sidsærk. English']
Author ['Aanrud, Hans, 1863-1953']
EBook-No. ['26348']
Release Date ['Aug 18, 2008']
Title ['Lisbeth Longfrock']
LoC Class ['PZ: Language and Litera

In [7]:
def get_list_link(page):
    links = []
    pgdbbyauthor = page.find("div", {"class": "pgdbbyauthor"})
    for element in pgdbbyauthor.children:
        if element.name == 'h2':
            pass
        if element.name == 'ul':
            for link in element.find_all('a'):
                link_key = link['href']
                if 'ebooks' in link_key:
                    links.append({'https://www.gutenberg.org' + link_key: link.text})
    return links

In [10]:
links

[]

In [11]:
def create_file(text, path):
    if os.path.isfile(path) == False:
        with open(path + '.txt', 'w') as file: 
            file.write(text) 

In [12]:
def create_dir(name):
    if os.path.exists(name) == False:
        os.mkdir(name)

In [13]:
def get_link_for_download(url):
    content = get_content(url)
    elements = BeautifulSoup(content, 'html.parser').find('table').find_all('tr')
    for element in elements:
        for attribute in element.attrs:
            if attribute == 'about':
                if '.txt' in element['about']:
                    return element['about']

In [14]:
def create_file_by_linc(link, dir_name):
    key = list(link.keys())[0]
    keu_for_download = get_link_for_download(key)
    r = requests.get(keu_for_download, stream=True)
    if r.status_code == 200:
        with open(dir_name + '/' + link[key], 'wb') as f:
            for chunk in r:
                f.write(chunk)
    #urllib.request.urlretrieve(keu_for_download, dir_name + '/' + link[key])

In [15]:
def download_files(links):
    dir_name = 'A'
    create_dir(dir_name)
    for link in links:
        create_file_by_linc(link, dir_name)
    #ThreadPool(9).imap_unordered(create_file_by_linc, links)

In [16]:
download_files(links)